In [1]:
import numpy as np

from pyboolnet.external.bnet2primes import bnet_text2primes
from pyboolnet.state_transition_graphs import primes2stg

from transition_matrix import get_transition_matrix
from scc_dags import get_scc_dag, get_attractor_states
from basins import get_strong_basins, compare_strong_basins
from matrix_operations import compress_matrix
from grouping import sd_grouping, null_grouping

In [2]:
DEBUG = True

np.set_printoptions(linewidth=1000, precision=3, suppress=True)

In [3]:
bnet = """
A, A | B & C
B, B & !C
C, B & !C | !C & !D | !B & C & D
D, !A & !B & !C & !D | !A & C & D
"""

update = "asynchronous"

primes = bnet_text2primes(bnet)
primes = {key: primes[key] for key in sorted(primes)}
stg = primes2stg(primes, update)

transition_matrix = get_transition_matrix(stg, update=update, DEBUG=DEBUG)

print(transition_matrix)

[[0.5  0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.75 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.75 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.75 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.5  0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25 0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.25]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.75 0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.75 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.75 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.75 0.   0.   0.   0.  ]
 [0.

In [ ]:
scc_dag = get_scc_dag(stg)
attractor_indices = get_attractor_states(scc_dag, as_indices=True, DEBUG=DEBUG)

print(attractor_indices)

[[0, 1, 2], [3], [8, 10]]


In [ ]:
strong_basin = get_strong_basins(transition_matrix, attractor_indices, exclude_attractors=True, DEBUG=DEBUG)

print(strong_basin)

[[-2.]
 [-2.]
 [-2.]
 [-2.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]]


In [ ]:
sd_indices = sd_grouping(bnet, DEBUG=DEBUG)

print(sd_indices)

Tm = compress_matrix(transition_matrix, sd_indices, DEBUG=DEBUG)

print("Tm")
print(np.round(Tm, 3))


[[4, 5, 6, 7], [], [0, 1, 2], [3], [12, 14], [8, 10], [13, 15], [9, 11]]
Tm
[[0.75  0.062 0.062 0.062 0.    0.062 0.   ]
 [0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.875 0.125 0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.25  0.    0.625 0.125]
 [0.    0.    0.    0.    0.25  0.    0.75 ]]


In [ ]:
sd_strong_basin = get_strong_basins(Tm, attractor_indices, grouped=True, group_indices=sd_indices, exclude_attractors=True, DEBUG=DEBUG)

print(sd_strong_basin)

[[-2.]
 [-2.]
 [-2.]
 [-2.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [ 2.]
 [-2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]]


In [8]:
compare_strong_basins(strong_basin, sd_strong_basin, DEBUG=DEBUG)

(np.int64(6), np.int64(0), np.int64(4), np.int64(0))

In [ ]:
null_indices = null_grouping(bnet, DEBUG=DEBUG)

print(null_indices)

T0 = compress_matrix(transition_matrix, null_indices, DEBUG=DEBUG)

print("T0")
print(np.round(T0, 3))

[[0, 1, 2, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15], [3], [8, 10]]
T0
[[0.923 0.019 0.058]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]


In [ ]:
null_strong_basin = get_strong_basins(T0, attractor_indices, grouped=True, group_indices=null_indices, exclude_attractors=True, DEBUG=DEBUG)

print(null_strong_basin)

[[-2.]
 [-2.]
 [-2.]
 [-2.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-1.]
 [-2.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]


In [11]:
compare_strong_basins(strong_basin, null_strong_basin, DEBUG=DEBUG)

(np.int64(0), np.int64(0), np.int64(4), np.int64(6))